In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Sep__8_19:56:38_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.3, V12.3.52
Build cuda_12.3.r12.3/compiler.33281558_0
torch:  2.1 ; cuda:  2.1.1
detectron2: 0.6


In [2]:
# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode,GenericMask,Visualizer
from detectron2.structures.keypoints import heatmaps_to_keypoints
from detectron2.structures.keypoints_hack import heatmaps_to_keypoints2, heatmaps_to_keypoints_iterative, heatmaps_modify_iterative
from matplotlib import pyplot as plt
import glob
import random
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    DatasetEvaluator,
    inference_on_dataset,
    print_csv_format,
    verify_results,
)
import cv2 

In [3]:
cd C:\R_projects\deep_learning_playground\detectron2\custom_data3

C:\R_projects\deep_learning_playground\detectron2\custom_data3


In [4]:
register_coco_instances("cat_data_train", {}, ".\coco_train.json", ".\datasets\sample1")
register_coco_instances("cat_data_test", {}, ".\coco_test.json", ".\datasets\sample1")
#register_coco_instances("cat_data_val", {}, ".\valid.json", ".\datasets\caterpillar")
MetadataCatalog.get("cat_data_train").set(thing_classes=["cat"])
MetadataCatalog.get("cat_data_train").set(thing_colors=(0,0,255))
MetadataCatalog.get("cat_data_test").set(thing_classes=["cat"])
MetadataCatalog.get("cat_data_test").set(thing_colors=(0,0,255))
MetadataCatalog.get("cat_data_train").set(keypoint_names=["head","middle","tail"])
MetadataCatalog.get("cat_data_train").set(keypoint_flip_map=[])
MetadataCatalog.get("cat_data_test").set(keypoint_names=["head","middle","tail"])
MetadataCatalog.get("cat_data_test").set(keypoint_flip_map=[])

namespace(name='cat_data_test',
          json_file='.\\coco_test.json',
          image_root='.\\datasets\\sample1',
          evaluator_type='coco',
          thing_classes=['cat'],
          thing_colors=(0, 0, 255),
          keypoint_names=['head', 'middle', 'tail'],
          keypoint_flip_map=[])

In [5]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.engine import DefaultTrainer

class CustomTrainer(DefaultTrainer):
    """
    Custom Trainer deriving from the "DefaultTrainer"

    Overloads build_hooks to add a hook to calculate loss on the test set during training.
    """

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1, LossEvalHook(
            400, # Frequency of calculation - every 100 iterations here
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg, True)
            )
        ))

        return hooks


In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cat_data_train",)
cfg.DATASETS.TEST = ("cat_data_test", )
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.DATALOADER.NUM_WORKERS = 2
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.0001  
cfg.SOLVER.MAX_ITER = 8000    
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.03 # Set to lower?
cfg.TEST.DETECTIONS_PER_IMAGE = 1
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 3
cfg.TEST.KEYPOINT_OKS_SIGMAS = [0.1, 0.3, 0.1]
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.MODEL.MASK_ON = True
cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.LOSS_WEIGHT = 5
cfg.MODEL.ROI_KEYPOINT_HEAD.NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS = False
#cfg.MODEL.ROI_KEYPOINT_HEAD.ITERATIVE = True
cfg.INPUT.USE_DIFF = False                      # Include difference in green channel 10 frames ago as the 4th channel
cfg.OUTPUT_DIR = "./modelv6"
#cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.INPUT.MAX_SIZE_TEST = 1000
cfg.INPUT.MAX_SIZE_TRAIN = 1000
cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MIN_SIZE_TRAIN = 1000

cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 5000
cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 3000
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 2500
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 2000
cfg.MODEL.RPN.NMS_THRESH = 0.7


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [7]:
print(cfg)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 2
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('cat_data_test',)
  TRAIN: ('cat_data_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1000
  MAX_SIZE_TRAIN: 1000
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: 1000
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
  USE_DIFF: False
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32], [64], [128], [256], [512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_fpn_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: ['res2', 'res3', '

In [8]:
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard --logdir cfg.OUTPUT_DIR

In [9]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0007999.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[01/18 14:31:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./modelv6\model_0007999.pth ...


In [10]:
# For writing ground truths and predictions 
import csv
from datetime import datetime

def write_csv(path, df):
    keys = df[0].keys()
    with open(path, "w", newline = "") as output_file: 
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(df)

def extract_pred_annotations(predictor, fi, data_name):
    inst = predictor(cv2.imread(fi))["instances"].to("cpu")
    data = {}
    data["file_name"] = fi
    data["image_size"] = list(inst.image_size)
    try: data["thing_class"] = MetadataCatalog.get(data_name).get("thing_classes")[inst.pred_classes[0]]
    except: data["thing_class"] = "NA"
    try: data["score"] = inst.scores.tolist()[0]
    except: data["score"] = "NA"
    try: data["keypoints"] = np.array(inst.pred_keypoints.tolist()[0]).ravel().tolist()
    except: data["keypoints"] = "NA"
    try: data["bbox"] = inst.pred_boxes.tensor.tolist()[0]
    except: data["bbox"] = "NA"
    try:
        cont, _ = cv2.findContours(inst.pred_masks.numpy()[0,:,:].astype('uint8'),cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        data["polygon"] = cont[0].ravel().tolist()
    except: 
        data["polygon"] = "NA"
    data["type"] = "prediction"
    return(data)

def extract_gt_annotations(data_dict, data_name):
    data = {}
    data["file_name"] = data_dict["file_name"]
    data["image_size"] = [data_dict["width"], data_dict["height"]]
    
    ann = data_dict["annotations"][0] # One instance for now
    
    try: data["thing_class"] = MetadataCatalog.get('cat_data_test').get("thing_classes")[ann["category_id"]]
    except: data["thing_class"] = "NA"
    data["score"] = "NA"
    try: data["keypoints"] = ann["keypoints"]
    except: data["keypoints"] = "NA"
    try: data["bbox"] = ann["bbox"]
    except: data["bbox"] = "NA"
    try: data["polygon"] = ann["segmentation"][0]
    except: data["polygon"] = "NA"
    data["type"] = "ground_truth"
    return(data)
    

def img_inference2(predictor, data_name, inference_info = ""):
    d = DatasetCatalog.get(data_name)
    dataset = []
    tot = len(d)
    for i in range(tot):
        print(f"{i+1} of {tot}", end = "\r")
        fi = d[i]["file_name"]
        data = extract_pred_annotations(predictor, fi, data_name)
        data["inference_info"] = inference_info
        dataset.append(data)
        data = extract_gt_annotations(d[i], data_name)
        data["inference_info"] = inference_info
        dataset.append(data)
    return(dataset)

def write_img_inference2(predictor, write_path, file_name_ID, data_name, model_ver):
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    inference_info = model_ver + "__" + now
    df = img_inference2(predictor, data_name, inference_info)
    write_csv(os.path.join(write_path, file_name_ID + "_inference.csv"), df)

In [11]:
# For writing predictions
import csv

def write_csv(path, df):
    keys = df[0].keys()
    with open(path, "w", newline = "") as output_file: 
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(df)

def img_inference(predictor, rep_ID, root_path, inference_info):
    dataset = []
    f = glob.glob(os.path.join(root_path, rep_ID,"*[!_diff].jpg"))
    tot = len(f)
    for i in range(tot):
        print(f"{i+1} of {tot}", end = "\r")
        fi = f[i]
        inst = predictor(cv2.imread(fi))["instances"].to("cpu")
        data = {}
        data["file_name"] = fi
        data["image_size"] = list(inst.image_size)
        try: data["thing_class"] = MetadataCatalog.get('cat_data_test').get("thing_classes")[inst.pred_classes[0]]
        except: data["thing_class"] = "NA"
        try: data["score"] = inst.scores.tolist()[0]
        except: data["score"] = "NA"
        try: data["keypoints"] = np.array(inst.pred_keypoints.tolist()[0]).ravel().tolist()
        except: data["keypoints"] = "NA"
        try: data["bbox"] = inst.pred_boxes.tensor.tolist()[0]
        except: data["bbox"] = "NA"
        try:
            cont, _ = cv2.findContours(inst.pred_masks.numpy()[0,:,:].astype('uint8'),cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            data["polygon"] = cont[0].ravel().tolist()
        except: 
            data["polygon"] = "NA"
        data["inference_info"] = inference_info
        dataset.append(data)
    return(dataset)

def write_img_inference(predictor, write_path, rep_ID, read_path, model_ver):
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    inference_info = model_ver + "__" + now
    df = img_inference(predictor, rep_ID, read_path, inference_info)
    write_csv(os.path.join(write_path, rep_ID + "_inference.csv"), df)

In [12]:
write_img_inference2(
    predictor, 
    write_path = "C:/R_projects/deep_learning_playground",
    file_name_ID = "sample1_modelv6",
    data_name = "cat_data_test", 
    model_ver = "modelv6"
    )

[01/18 14:31:17 d2.data.datasets.coco]: Loaded 123 images in COCO format from .\coco_test.json


C:\Users\vsbpa\.conda\envs\detectron_env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [13]:
rep_IDs = [os.path.basename(x) 
 for x in 
 glob.glob("C:/R_projects/spat_1f_noise/processed_feed/rep*", recursive=False)]

In [16]:
for ID in rep_IDs:
    print(f"Processing {ID}", end = "\n")
    write_img_inference(
    predictor, 
    write_path = "C:/R_projects/deep_learning_playground/inferences",
    rep_ID = ID,
    read_path = "C:/R_projects/spat_1f_noise/processed_feed", 
    model_ver = "modelv6"
    )
  

Processing rep1
Processing rep10
Processing rep100
Processing rep101
Processing rep102
Processing rep103
Processing rep104
Processing rep105
Processing rep106
Processing rep107
Processing rep108
Processing rep109
Processing rep11
Processing rep110
Processing rep111
Processing rep112
Processing rep113
Processing rep114
Processing rep115
Processing rep116
Processing rep117
Processing rep118
Processing rep119
Processing rep12
Processing rep120
Processing rep121
Processing rep129
Processing rep13
Processing rep130
Processing rep131
Processing rep132
Processing rep133
Processing rep134
Processing rep135
Processing rep136
Processing rep137
Processing rep138
Processing rep139
Processing rep14
Processing rep140
Processing rep141
Processing rep142
Processing rep143
Processing rep144
Processing rep145
Processing rep146
Processing rep147
Processing rep148
Processing rep149
Processing rep15
Processing rep150
Processing rep151
Processing rep152
Processing rep153
Processing rep154
Processing rep155
